## crawler data

In [7]:
import requests
import os
import zipfile
import time
import datetime

def real_estate_crawler(year, season):
  if year > 1000:
    year -= 1911

  # download real estate zip content
  res = requests.get("https://plvr.land.moi.gov.tw//DownloadSeason?season="+str(year)+"S"+str(season)+"&type=zip&fileName=lvr_landcsv.zip")

  # check if there's error return, if not, continue
  try:
    if '系統訊息' in res.content.decode():
      pass
  except:
      # save content to file
      fname = str(year)+str(season)+'.zip'
      open(fname, 'wb').write(res.content)

      # make additional folder for files to extract
      folder = 'real_estate' + str(year) + str(season)
      if not os.path.isdir(folder):
        os.mkdir(folder)  

      # extract files to the folder
      with zipfile.ZipFile(fname, 'r') as zip_ref:
          zip_ref.extractall(folder)
      time.sleep(5)
      #delete zip file
      os.remove(fname)
      
def get_folder_name():
    folder_name = os.listdir()
    return [x.split('real_estate')[1] for x in folder_name if x.startswith('real_estate')]

def get_file():
    #get current folder name
    current_list=get_folder_name()
    print(f'current list: {current_list}')
    #get current year in TW format
    current_year = datetime.datetime.now().year -1911
    
    for year in range(112, current_year+1):
        """which year range you want to check"""
        for season in range(1,5):
            ## check if already had extract files
            if str(year)+str(season) not in current_list:
                print(f' now in year:{year} and quarter: {season}')
                real_estate_crawler(year, season)
    print('Get data complete.')

In [ ]:
get_file()

## Get data



In [50]:
import os
import pandas as pd

# 歷年資料夾
dirs = [d for d in os.listdir() if d[:4] == 'real']

dfs = []

for d in dirs:
    print(d)
    df = pd.read_csv(os.path.join(d,'b_lvr_land_a.csv'), index_col=False)
    df['Q'] = d[-1]
    dfs.append(df.iloc[1:])
    
df = pd.concat(dfs, sort=True)
df.reset_index(inplace=True, drop=True)

real_estate1061
real_estate1062
real_estate1063
real_estate1064
real_estate1071
real_estate1072
real_estate1073
real_estate1074
real_estate1081
real_estate1082
real_estate1083
real_estate1084
real_estate1091
real_estate1092
real_estate1093
real_estate1094


C:\Users\yipin\AppData\Local\Temp\ipykernel_8408\2235116703.py:11: DtypeWarning: Columns (3,7,14,15,16,17,18,21,22,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(d,'b_lvr_land_a.csv'), index_col=False)
C:\Users\yipin\AppData\Local\Temp\ipykernel_8408\2235116703.py:11: DtypeWarning: Columns (3,7,14,15,16,17,18,21,22,24,25,28,29,30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(d,'b_lvr_land_a.csv'), index_col=False)


real_estate1101
real_estate1102
real_estate1103
real_estate1104
real_estate1111
real_estate1112
real_estate1113
real_estate1114
real_estate1121


### Data pre-processing

In [51]:
def drop_nan_columns(df, columns):
    ## input a dataframe assign a columns name, drop columns with all nan values base on certain columns
    for column in columns:
        df = df.dropna(subset=[column])
    return df

df = drop_nan_columns(df, ['單價元平方公尺'])
# 不同名稱同項目資料合併
# df['單價元平方公尺'].fillna(df['單價元/平方公尺'], inplace=True)
# df.drop(columns='單價元/平方公尺')

# 建物型態
df['建物型態2'] = df['建物型態'].str.split('(').str[0]
df.drop(columns=['建物型態'],axis=1,inplace=True)

# 刪除有備註之交易（多為親友交易、價格不正常之交易）
df = df[df['備註'].isnull()]

## give some list and drop columns
df = df.drop(['備註','移轉編號','編號','非都市土地使用分區','非都市土地使用編定'], axis=1)

##交易筆棟數 change to 土地 建物 車位
## a datafram column 交易筆棟數 element format is 土地1建物1車位1, I want to split to three columns which is ['土地','建物','車位',] element is 1, 1, 1
df['土地'] = df['交易筆棟數'].str.split('土地',expand=True)[1].str.split('建物',expand=True)[0]
df['建物'] = df['交易筆棟數'].str.split('建物',expand=True)[1].str.split('車位',expand=True)[0]
df['車位'] = df['交易筆棟數'].str.split('車位',expand=True)[1]
df.drop(columns=['交易筆棟數'],axis=1,inplace=True)

#split to solely 
df['交易標的'] = df['交易標的'].str.split('(',expand=True)[0]

## rename
df.rename(columns={'建物現況格局-廳': '廳數', '建物現況格局-房':'房','建物現況格局-衛':'衛'}, inplace=True)

#some data form is corrupted need to drop in 交易年月日 and 建築完成年月
df = df[df['交易年月日'].astype(str).apply(lambda x:(x.split('.')[0][-4:]))!='0000']
df = df[df['建築完成年月'].astype(str).apply(lambda x:(x.split('.')[0][-2:]))!='00']
df = df[df['建築完成年月'].astype(str).apply(lambda x:(x.split('.')[0][-4:-2]))!='00']
# space in this column cause error
df = df[df['建築完成年月'].apply(lambda x:(' ' not in str(x)))]
df = df[df['建築完成年月'].astype(str).apply(lambda x:(len(x)))>5]

#translate to date format
df['year'] = df['交易年月日'].astype(str).str[:-4].astype(int) + 1911
df['trade_date'] = pd.to_datetime(dict(year=df['year'], month=df['交易年月日'].astype(str).apply(lambda x:(x[-4:-2])), day=df['交易年月日'].astype(str).apply(lambda x:(x[-2:]))))

df['build_year'] = df['建築完成年月'].astype(str).apply(lambda x:(x.split('.')[0][:-4])).astype(int) + 1911
df=df[df['build_year']<2200]

## set all day to 1 because encounter some month misatch with day
df['build_date'] = pd.to_datetime(dict(year=df['build_year'], month=df['建築完成年月'].astype(str).apply(lambda x:(x.split('.')[0][-4:-2])), day=1))


df.drop(columns=['建物現況格局-隔間','year','build_year','Q','交易年月日','建築完成年月',],axis=1,inplace=True)


### filter data

In [52]:
#exclude only 土地
df = df[df['交易標的']!='土地']

#建物型態2 to only 華夏 住宅大樓 公寓 透天
df = df[df['建物型態2'].isin(['住宅大樓','華廈','透天厝','公寓'])]

#exclude 商業工業用... but many is 見其他登記事項
df = df[df['主要用途'].apply(lambda x:(('住' in str(x)) or ('見其他' in str(x))))]


### Data quality check

In [53]:
# https://w3fs.tainan.gov.tw/Download.ashx?u=LzAwMS9VcGxvYWQvMjA3L3JlbGZpbGUvMjI0ODYvNzgxMTE1Ni8yZGMzOWViYi01OTMzLTRmMGUtOTYzMC1kMTA4ZmY5YjAxMzIucGRm&n=MjAxNTEyMTExNTAyNDk5OTc2MTgucGRm&icon=.pdf
# as mentioned above, there's change that housing data would be corrupted

# 平方公尺換成坪


def translate_m2(df=''):
    """
    Input parameters:
        Dataframe from house extract
    Return:
        Dataframe translate from 平方公尺 to 坪
    """
    if len(df)==0:
        raise ValueError('No Dataframe input')
    trans_dict = {'主建物面積':'主建物面積', '土地移轉總面積平方公尺':'土地面積','建物移轉總面積平方公尺':'建物面積',
                '車位移轉總面積(平方公尺)':'車位','附屬建物面積':'附屬建物面積','陽台面積':'陽台面積','單價元平方公尺':'單價元坪',}
    if len(list(df.columns)) != len(list(set(list(trans_dict.keys()) + list(df.columns)))):
        raise ValueError('Missing required columns')
    
    for key, item in trans_dict.items():
        df[item] = df[key].astype(float) * 3.30579
        if item!=key:
            df.drop(columns=[key],axis=1,inplace=True)
    return df

df = translate_m2(df)

#https://www-ws.land.ntpc.gov.tw/001/Upload/oldFile/userfiles/FE/file/0725%e5%af%a6%e5%83%b9%e7%99%bb%e9%8c%84%e6%96%b0%e5%8a%9f%e8%83%bd%20%e5%bb%ba%e5%9d%aa%e5%96%ae%e5%83%b9%e6%96%b0%e7%ae%97%e6%b3%95%20%e9%81%bf%e5%85%8d%e8%99%9b%e5%9d%aa.pdf
#explain on column 主建物面積 can be use for caculate 公設比



### Save data

In [4]:
df.to_csv('holi_fonuan.csv', index=False, encoding='utf-8-sig')

In [ ]:
# 將index改成年月日
# df = drop_nan_columns(df, ['交易年月日'])
# df.index = pd.to_datetime((df['交易年月日'].str[:-4].astype(int) + 1911).astype(str) + df['交易年月日'].str[-4:] ,errors='coerce')

In [ ]:
df[['主要用途','廳數']].groupby(['主要用途']).size().loc[lambda x: x>10].sort_values(ascending=False).reset_index()

In [146]:
df.reset_index(inplace=True, drop=True)

In [54]:
df.columns

Index(['主建物面積', '主要建材', '主要用途', '交易標的', '土地位置建物門牌', '廳數', '房', '衛', '有無管理組織',
       '移轉層次', '總價元', '總樓層數', '車位總價元', '車位類別', '都市土地使用分區', '鄉鎮市區', '附屬建物面積',
       '陽台面積', '電梯', '建物型態2', '土地', '建物', '車位', 'trade_date', 'build_date',
       '土地面積', '建物面積', '單價元坪'],
      dtype='object')

In [ ]:
columns = ['建物面積', '土地面積',
       '建物移轉總面積平方公尺', '移轉層次', '總價元',
       '總樓層數', '車位', '車位總價元', '車位類別', '附屬建物面積',
       '陽台面積', '單價元坪', '建物型態2', '土地', '建物', '車位', '土地位置建物門牌','trade_date',]
df[columns]